In [1]:
from ultralytics import YOLO
import ultralytics
import glob
from IPython.display import Image, display
import random
import torch

In [2]:
print(torch.__version__)
print(ultralytics.__version__)

2.4.1+cu121
8.3.105


In [3]:
from pathlib import Path
from PIL import Image

images_path = Path("datasets/images")  # đổi lại path nếu cần
for img_path in images_path.glob("**/*.JPG"):
    try:
        with Image.open(img_path) as img:
            img.verify()  # chỉ kiểm tra chứ không load
    except Exception as e:
        print(f"❌ Error in {img_path}: {e}")


In [4]:
from pathlib import Path

def is_valid_label_line(line):
    parts = line.strip().split()
    if len(parts) != 5:
        return False
    try:
        class_id = int(parts[0])
        if class_id < 0:
            return False
        values = list(map(float, parts[1:]))
        return all(0.0 <= val <= 1.0 for val in values)
    except:
        return False

labels_path = Path("datasets/labels")  # đổi path nếu cần
error_files = []

for label_file in labels_path.glob("**/*.txt"):
    with open(label_file, "r") as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if not is_valid_label_line(line):
                error_files.append((label_file, i+1, line.strip()))

if error_files:
    print("❌ Lỗi định dạng trong các file sau:")
    for file, line_num, content in error_files:
        print(f"File: {file}, dòng {line_num}: {content}")
else:
    print("✅ Tất cả file label đều đúng định dạng YOLO.")

✅ Tất cả file label đều đúng định dạng YOLO.


In [ ]:
model = YOLO("yolo11s.pt")  

# Bắt đầu huấn luyện
model.train(
    data="./data.yaml",    # đường dẫn đến file .yaml
    epochs=30,           # số epoch
    imgsz=640,           # kích thước ảnh
    batch=16,
    device='cpu'   
)

In [ ]:
model = YOLO("runs/detect/train450/weights/best.pt")
# model.model.names = {
#     0: 'g',
#     1: 'ng'
# }

In [4]:
test_img_paths = []
for _ in range(10):
    rand_num = random.randint(200, 899)
    rand_num = str(rand_num).zfill(4)
    test_img_paths.append(f"D:/object detection nn/rice_seed_data/IMG_{rand_num}.JPG")

In [5]:
test_img_paths

['D:/object detection nn/rice_seed_data/IMG_0646.JPG',
 'D:/object detection nn/rice_seed_data/IMG_0654.JPG',
 'D:/object detection nn/rice_seed_data/IMG_0887.JPG',
 'D:/object detection nn/rice_seed_data/IMG_0245.JPG',
 'D:/object detection nn/rice_seed_data/IMG_0725.JPG',
 'D:/object detection nn/rice_seed_data/IMG_0669.JPG',
 'D:/object detection nn/rice_seed_data/IMG_0443.JPG',
 'D:/object detection nn/rice_seed_data/IMG_0300.JPG',
 'D:/object detection nn/rice_seed_data/IMG_0494.JPG',
 'D:/object detection nn/rice_seed_data/IMG_0810.JPG']

In [6]:
results = model(test_img_paths, save=True, conf=0.25)


0: 448x640 10 gs, 29 ns, 9.4ms
1: 448x640 24 gs, 5 ns, 9.4ms
2: 448x640 36 gs, 22 ns, 9.4ms
3: 448x640 30 gs, 11 ns, 9.4ms
4: 448x640 19 gs, 18 ns, 9.4ms
5: 448x640 20 gs, 9 ns, 9.4ms
6: 448x640 34 gs, 9 ns, 9.4ms
7: 448x640 22 gs, 37 ns, 9.4ms
8: 448x640 26 gs, 4 ns, 9.4ms
9: 448x640 3 gs, 32 ns, 9.4ms
Speed: 2.5ms preprocess, 9.4ms inference, 19.5ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict
